# langfuse prompt engineering 

In [2]:
# import
from langfuse import Langfuse
import openai
import os 
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-4624705e-b895-41ec-9a26-4012c6274411"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-1f991509-4342-4860-a304-ebb8198a5d24"
os.environ["LANGFUSE_HOST"] = "http://localhost:3000"


from langfuse import Langfuse
langfuse = Langfuse()
 
# optional, verify that Langfuse is configured correctly
langfuse.auth_check()

In [12]:
langfuse.create_prompt(
    name="post_summarization",
    prompt="해당 텍스트에서 주요 정보를 추출하여 주어진 JSON의 데이터를 반환하시오 . Json 스키마는 이와 같다: {{json_schema}}",
    config={
        "model":"gpt-3.5-turbo-1106",
        "temperature": 0,
        "json_schema":{
            "date": "게시글 업로드 날짜 ",
            "content": "한문장으로 구성된 요약 내용",
            "keywords": "array 로 된 키워드",
            "category": "건의, 자유, 버그 중 하나",
            "sentiment": "-1 과 1 사이의 sentiment score",
            "game": "관련 게임 명"
        }
    },
    labels=["production"]
);

In [13]:
prompt = langfuse.get_prompt("post_summarization")

In [14]:
prompt.config

{'model': 'gpt-3.5-turbo-1106',
 'json_schema': {'date': '게시글 업로드 날짜 ',
  'game': '관련 게임 명',
  'content': '한문장으로 구성된 요약 내용',
  'category': '건의, 자유, 버그 중 하나',
  'keywords': 'array 로 된 키워드',
  'sentiment': '-1 과 1 사이의 sentiment score'},
 'temperature': 0}

In [20]:
from langfuse.openai import OpenAI
client = OpenAI()

In [21]:
import json
 
def summarize_post(story):
  # Stringify the JSON schema
  json_schema_str = ', '.join([f"'{key}': {value}" for key, value in prompt.config["json_schema"].items()])
 
  # Compile prompt with stringified version of json schema
  system_message = prompt.compile(json_schema=json_schema_str)
 
  # Format as OpenAI messages
  messages = [
      {"role":"system","content": system_message},
      {"role":"user","content":story}
  ]
 
  # Get additional config
  model = prompt.config["model"]
  temperature = prompt.config["temperature"]
 
  # Execute LLM call
  res = client.chat.completions.create(
    model = model,
    temperature = temperature,
    messages = messages,
    response_format = { "type": "json_object" },
    langfuse_prompt = prompt # capture used prompt version in trace
  )
 
  # Parse response as JSON
  res = json.loads(res.choices[0].message.content)
 
  return res

In [22]:
summarize_post(post)

{'date': '2024.09.22',
 'game': '찍찍이약먹어추천',
 'content': '보호조치, 영구정지, 작업장(계정 무한반복 삭제) 문제심각. 게임사에 유저들이 건의한 내용을 취합하여 남깁니다. 메인퀘스트 셀로비아, 흐니르에서 드랍되는 거인템+미결은 니다벨리르 이후 메인퀘스트 보스 드랍템으로 변경, 파티던전 맹독의뱀 매주 3회 무료이용권으로 드랍되는템은 귀속으로 변경, 신규유저 희귀장비 지급일을 5일 이후로 변경, 거래소 판매 렙제한 40이상으로 변경, 매주 계정 세탁 초기화방지 수단으로 오딘 계정 탈퇴한 카카오톡으로 1주일간 재가입 불가 딜레이만 걸어주셔도 됩니다. 게임속 작업장이 있던 말던 상관없습니다만, 게임 시스템 악용하고 있는 어뷰징만은 막아주리라 믿습니다.(계정초기화)',
 'category': '건의',
 'keywords': ['찍찍이약먹어추천',
  '보호조치',
  '영구정지',
  '작업장',
  '게임사',
  '유저들',
  '건의',
  '메인퀘스트',
  '파티던전',
  '신규유저',
  '거래소',
  '계정 세탁 초기화방지',
  '작업장 수익구조'],
 'sentiment': -0.3}

In [26]:
summarize_post(post2)

{'date': '2024.09.22',
 'game': '니플하임',
 'content': '니플하임 초반 몬스터 피통을 깍아주고 이번 무스펠 9강 공컬렉 나온 것처럼 7강 이상이여도 유효컬렉(공 명) 이렇게 분포를 해주시면 좋을거같습니다',
 'category': '건의',
 'keywords': ['니플하임', '개편', '투력', '사냥터', '컬렉', '경험치', '골드'],
 'sentiment': 0.5}

In [25]:
post2 = """
니플하임 개편 부탁드립니다 (모든 7~8만투 공감내용)

굴베장신추천 0조회 48324.09.22 16:53댓글 4북마크공유하기기능 더보기
모든 7~8만투가 공감하는 내용입니다
이투력에선 투력을 올리기도 쉽지 않은데
사냥터는 니플하임 초반이 메리트가 크게 없어 그대로 아스가르드 마지막 사냥터에서 사냥을 합니다

투력구간에서 자주 하는말들이 니플하임 초반가면 몬스터들이 아스 마지막보다 확 단단해지고 피통도 늘어나서
사냥속도도 오래걸리며, 그에 맞는 컬렉도 유효 컬렉이아닌 막기 막기상쇄 이런 부주력의 컬렉들이라

차라리 경험치 골드 더주는 격전으로 몰리고 있습니다,,

니플하임 초반 몬스터 피통을 깍아주고 이번 무스펠 9강 공컬렉 나온 것처럼 7강 이상이여도 유효컬렉(공 명) 이렇게 분포를 해주시면 좋을거같습니다

심지어 아스 초입에도 유효 컬렉 아이템 파밍이 가능합니다.

이상입니다
"""

In [19]:
post = """ 찍찍이약먹어추천 0조회 7024.09.22 21:12댓글 0북마크공유하기기능 더보기
기존 내용 참조

https://cafe.daum.net/odin/DEIV/35872


[재업]보호조치, 영구정지, 작업장(계정 무한반복 삭제) 문제심각

*2024-03-25추가현재 이슈되고 있는 유튜버 무끝님 영구정지로 인해 다시 한번 업로드 합니다.플레이어 여러분이 보호조치나, 영구정지로 무고밴 당했다면 그건 작업장이랑 같은 루트로 된 미드-

cafe.daum.net
 

 

 

 

미드-요툰 흐니르 까지 미는 작업장을 방치하는 이유가 뭘까요?

마켓 다운로드수 때문일까요? 아니면 작업장 유입으로 인한 접속수 일까요?

저번주 업뎃이트로 신규서버에 유입되는 신규유저 눈속임으로 거미알 터트리기가 단일이 아닌 중복으로 가능하게끔 패치하셧더군요.

보다 쾌적한 게임환경을 위해 미드-요툰 흐니르 까지 미는 작업장 단속 안하는게 니다벨리르 몬스터 개체수를 줄이는것보다 더 효율적이 였을까요?

 

일부 골빈유저들이 이런얘기를 하는데 저런 작업장이 있어 재료템을 저렴하게 산다고? 

다시 한번 말하자만 쟤네 주수입원은 흐니르에서 얻어가는 미결+파티던전에서 나오는 미결입니다.

거인템 시세 하락으로 주수입원은 그냥 미결뿐입니다.

지하감옥 난쟁이에서 드랍되는 위그드람 구드벨 각종강화석은 쟤네랑 1도 상관없어요.

매주 월요일에 계정초기화 작업후 매 유저수가 많은 도시섭(오딘+토르+신규서버 에기르)에 캐릭생성후

한개서버마다 미결작업 하고 게임사에서 방치하는 주말 시간을 이용해 불법다이아 유통후 월요일 다시 계정 초기화 작업 반복하고 있습니다.

게임사에 유저들이 건의드린 내용은 그간 너무나 많은데 취합해 남겨드립니다.

1, 메인퀘스트 셀로비아, 흐니르 에서 드랍되는 거인템+미결은 니다벨리르 이후 메인퀘스트 보스 드랍템으로 변경

2, 파티던전 맹독의뱀 매주 3회 무료이용권으로 드랍되는템은 귀속으로 변경 

3, 신규유저 희귀장비 지급일을 5일 이후로 변경

4, 거래소 판매 렙제한 40이상으로 변경

 

추가

5, 매주 계정 세탁 초기화방지 수단으로 오딘 계정 탈퇴한 카카오톡으로 1주일간 재가입 불가 딜레이만 걸어주셔도 됩니다.

게임속 작업장이 있던 말던 상관없습니다만, 게임 시스템 악용하고 있는 어뷰징만은 막아주리라 믿습니다.(계정초기화)

이걸 반대하는건 그냥 작업장.

 

이중 하나라도 해주면 게시판 다시는 어지럽게 하지 않겠습니다.

그게 아니라면 처리 할때까지 매일 올리겠습니다.

 

저는 작업장이 벌어가는것을 게임사가 더 많이 벌어가길 원합니다.

이런작업장 수익구조를 원천차단 하고 그 수익구조를 게임사가 가져가는 패치를 하심을 추천드립니다. 

작업장이 없다고 해서 재료 가격이 오른다고 하시는데,

작업장이 벌어가는 수익을 게임사가 가져가는 패치를 하면 그에 따른 재료수급 루트를 넓혀주는게 게임사가 해야할일입니다.

 
저작자 표시컨텐츠변경비영리
댓글0추천해요0스크랩0 """